In [22]:
# from getopt import getopt
# import cloudpickle
# import pickle
# import sys
# import os
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.special import logit#, expit
from scipy.stats import uniform, norm, bernoulli, beta
# from statsmodels.stats.proportion import proportions_ztest
from matplotlib import pyplot as plt
# import pymc as pm
# import arviz as az
# from modeltools import mcmc_diagnostics, create_summary_stat
# from downcast import downcast_df
# import jax
# from pymc.sampling_jax import sample_numpyro_nuts
# from time import time, sleep
# from datetime import timedelta

## Checking whether loss is returned correctly

In [7]:
power_ds = pd.read_csv("data/simulations/power_powerv2.csv")
power_ds = power_ds.rename(columns={"cores_per_r":"scores_per_r"})

In [8]:
power_ds["propz_0_05"] = power_ds["propz_pval"]<0.05
power_ds["bht_0_05"] = power_ds["bht_pval"]<0.05

In [16]:
a = power_ds.groupby("sim_id").agg({"propz_0_05":"sum"})
b = power_ds.groupby("sim_id").agg({"bht_0_05":"sum"})
c = power_ds.groupby("sim_id").agg({"propz_0_05":"count"}).rename(columns={"propz_0_05":"total"})
d = pd.concat([a, b, c], axis="columns").reset_index()

In [14]:
logit(0.05)

-2.9444389791664403

In [17]:
d["loss"] = abs(logit(0.9) - logit(d["bht_0_05"]/d["total"]))

In [49]:
# For 10 success and 3 loss, 0.9 of the posterior distribution is less than 0.912
print(beta.ppf(0.9, 12, 2))
beta.ppf(0.2, 13, 1)

0.9158671129572205

In [57]:
N = 18
for N in range(12, 72, 12):
    for i in range(N):
        result = beta.cdf(0.9, i+1, N-i+1)
        if 0.025<result<0.975:
            print(f"Total:{N}, Success:{i}, 0.9 p_val:{result:.3f}")
    print("-----------------------------")

Total:12, Success:9, 0.9 p_val:0.966
Total:12, Success:10, 0.9 p_val:0.866
Total:12, Success:11, 0.9 p_val:0.621
-----------------------------
Total:24, Success:19, 0.9 p_val:0.967
Total:24, Success:20, 0.9 p_val:0.902
Total:24, Success:21, 0.9 p_val:0.764
Total:24, Success:22, 0.9 p_val:0.537
Total:24, Success:23, 0.9 p_val:0.271
-----------------------------
Total:36, Success:29, 0.9 p_val:0.973
Total:36, Success:30, 0.9 p_val:0.929
Total:36, Success:31, 0.9 p_val:0.840
Total:36, Success:32, 0.9 p_val:0.691
Total:36, Success:33, 0.9 p_val:0.486
Total:36, Success:34, 0.9 p_val:0.270
Total:36, Success:35, 0.9 p_val:0.104
-----------------------------
Total:48, Success:40, 0.9 p_val:0.948
Total:48, Success:41, 0.9 p_val:0.888
Total:48, Success:42, 0.9 p_val:0.785
Total:48, Success:43, 0.9 p_val:0.635
Total:48, Success:44, 0.9 p_val:0.450
Total:48, Success:45, 0.9 p_val:0.265
Total:48, Success:46, 0.9 p_val:0.120
Total:48, Success:47, 0.9 p_val:0.037
-----------------------------
Total:6

In [21]:
d[:40]

,sim_id,propz_0_05,bht_0_05,total,loss
0,0,10,10,12,0.587787
1,1,11,10,12,0.587787
2,2,22,20,24,0.587787
3,3,10,9,12,1.098612
4,4,9,6,12,2.197225
5,5,7,4,12,2.890372
6,6,23,24,24,inf
7,7,9,8,12,1.504077
8,8,11,9,12,1.098612
9,9,31,31,36,0.372675
